In [29]:
import pandas as pd

df = pd.read_excel('RAG/results/retrieval_file_7.xlsx')

questions = df.questions

In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb
client = chromadb.PersistentClient()

In [30]:
collection = client.get_collection('collection_1')

In [6]:
text = 'Vaccination remains most economical and effective means'
collection.get(where_document={'$contains': text})

{'ids': ['cf2f22a4-e52d-11ee-92f1-00155da004c0'],
 'embeddings': None,
 'metadatas': [{'source': 'data/covid/PMC3979752.xml.json.txt'}],
 'documents': ['Vaccination remains most economical and effective means against respiratory diseases caused by influenza viruses  . Based on the circulating viruses in the population, trivalent vaccine strains have been developed and are used for the influenza virus protection  . The most acceptable current available strategy is the intramuscular administration of inactivated vaccines produced by egg-based manufacturing systems which while effective, are hampered by limited capacity and flexibility  . However, vaccine strains must be frequently adapted to match the circulating viruses throughout the world  . In addition, the levels of antibody induced by the inactivated vaccine have been observed to decrease by 75% over an 8-month period  ,  . Therefore, alternative strategies for developing broadly'],
 'uris': None,
 'data': None}

In [17]:
collection.metadata

{'hnsw:space': 'cosine'}

In [31]:
model_name = 'sentence-transformers/msmarco-MiniLM-L-6-v3'

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer(model_name)

retrieved_docs = []
for q in questions:
    query_embedding = model.encode(q)
    res = collection.query(query_embeddings=[query_embedding.tolist()], include=['documents'])['documents']
    retrieved_docs.append(res)

In [35]:
retrieved_docs[2]

[['agents, the mechanisms of exacerbation may vary considerably in different individuals under various triggers. Acute exacerbations are usually due to the presence of environmental factors such as allergens, pollutants, smoke, cold or dry air and pathogenic microbes in the airway  . These agents elicit an immune response leading to infiltration of activated immune cells that further release inflammatory mediators that cause acute symptoms such as increased mucus production, cough, wheeze and shortness of breath. Among these agents, viral infection is one of the major drivers of asthma exacerbations accounting for up to 80–90% and 45–80% of exacerbations in children and adults respectively  . Viral involvement in COPD exacerbation is also equally high, having been detected in 30–80% of acute',
  'the exact mechanisms underpinning viral exacerbations in susceptible subjects in order to properly manage exacerbations via supplementary treatments that may alleviate the exacerbation symptom

In [33]:
df.retrieved_docs

0      ['mechanism  . Moreover, infiltrating cells in...
1      ['distress  , coronavirus, and human immunodef...
2      ['agents, the mechanisms of exacerbation may v...
3      ['delivery vector, led to the widespread testi...
4      ['to the empty vector may completely abrogate ...
                             ...                        
282    ['surveillance data are available for a popula...
283    ['pneumococci  , Bacillus anthracis  , Haemoph...
284    ['Mammarenaviruses   cause chronic infections ...
285    ['of disease symptoms.   In one instance, a HC...
286    ['was used to perform quantitative real-time P...
Name: retrieved_docs, Length: 287, dtype: object

In [22]:
from langchain_community.embeddings import FakeEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores.chroma import Chroma

def get_vectorstore(chroma_collection_name = 'collection_1',
                    distance_fn = 'cosine',
                    embedding_model = 'sentence-transformers/msmarco-MiniLM-L-6-v3'):
    
    # connect to Chroma client
    client = chromadb.PersistentClient()

    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    emb_model = HuggingFaceEmbeddings(model_name=model_name,
                                          model_kwargs=model_kwargs,
                                          encode_kwargs=encode_kwargs)

    # Langchain Chroma wrapper
    langchain_chroma = Chroma(client=client,
                              collection_name=chroma_collection_name,
                              embedding_function=emb_model,
                              collection_metadata={"hnsw:space": distance_fn})  
    print(f"{langchain_chroma._embedding_function.model_name=}")  
    print(f"{langchain_chroma._collection.name=}")  
    print(f"{langchain_chroma._collection.metadata=}")  
    return langchain_chroma

vector_store = get_vectorstore()

langchain_chroma._embedding_function.model_name='sentence-transformers/msmarco-MiniLM-L-6-v3'
langchain_chroma._collection.name='collection_1'
langchain_chroma._collection.metadata={'hnsw:space': 'cosine'}


In [25]:
vector_store.similarity_search(query=text, k=4, filter=None)

[Document(page_content='political will in low- and middle-income countries to commit to immunization as a priority, social marketing to individuals and communities, strengthening health systems and promoting relevant local research and development innovations  . Maternal vaccination to prevent disease in the youngest infants has been shown to be effective for tetanus, influenza and pertussis  . Influenza vaccination during pregnancy is safe, provides reasonable maternal protection against influenza, and also protects infants for a limited period from confirmed influenza infection  . However as antibody levels drop sharply after birth, infant protection does not persist much beyond 8 weeks  . Recently respiratory syncytial virus vaccination in pregnancy has been shown to be safe and immunogenic, and a phase-3', metadata={'source': 'data/covid/PMC5608782.xml.json.txt'}),
 Document(page_content='Vaccination remains most economical and effective means against respiratory diseases caused by